In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained('aisingapore/sealion3b', trust_remote_code = True)
model = AutoModelForCausalLM.from_pretrained(
    'aisingapore/sealion3b',
    trust_remote_code = True,
    torch_dtype = torch.float16,
    device_map="cuda:0"
)

[2023-11-11 10:55:52,482] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [5]:
from tqdm import tqdm
import os
import random
import json

In [6]:
questions = []
with open('quiz-tatabahasa.jsonl') as fopen:
    for no, l in enumerate(fopen):
        l = json.loads(l)
        soalan = [l['question']]
        jawapan = None
        for c, k in l['choices'].items():
            soalan.append(f"{c}. {k['text']}")
            if k['answer']:
                jawapan = c
        
        data = {
            'no': no,
            'objektif': 'Jawab soalan yang diberikan' if l['instruction'] is None else l['instruction'],
            'soalan': '\n'.join(soalan),
            'jawapan': jawapan,
        }
        questions.append(data)
len(questions)

349

In [7]:
arange = set(range(len(questions)))

In [8]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

In [10]:
for i in tqdm(range(len(questions))):
    
    prompts = []
    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
    inputs.pop('token_type_ids', None)
    repeat = []
    for _ in range(5):
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=3,
                top_p=0.95,
                top_k=50,
                temperature=0.5,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            r = tokenizer.decode(r[0]).split('jawapan:')[-1].strip().split()
            repeat.append(r[0].replace('.', '').replace('</s>', '').split('\\')[0].split('/')[0])
    
        except Exception as e:
            print(e)
            pass
    
    questions[i]['output'] = repeat

100%|██████████| 349/349 [02:37<00:00,  2.22it/s]


In [11]:
import json

with open('output-0shot-aisingapore-sealion3b.json', 'w') as fopen:
    json.dump(questions, fopen)

In [12]:
def most_common(l):
    return max(set(l), key=l.count)

In [13]:
filtered = [q for q in questions if 'output' in q]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

21.776504297994272